In [ ]:
import sys
dataFolder = '../../../../data/fields/'
sys.path.append(dataFolder)
from Auxiliary.helper import *

In [ ]:
os.chdir('../../../../data/')
img_lab_path = ['fields/Auxiliary/vrt/Force_X_from_68_to_69_Y_from_42_to_42_Cube.vrt']
## define function to load vrt
def loadVRTintoNumpy(vrtPath):
    ds = gdal.Open(vrtPath)
    bandNumber = ds.RasterCount
    bands = []
    for i in range(bandNumber):
        bands.append(ds.GetRasterBand(i+1).ReadAsArray())
    cube = np.dstack(bands)
    data_cube = np.transpose(cube, (2, 0, 1))
    reshaped_cube = data_cube.reshape(4, 6, 3000, 6000)
    return reshaped_cube


In [ ]:
# define the model (.pth) and assess loss curves
model_name = dataFolder + 'output/models/model_state_All_but_LU_transformed_42.pth'
model_name_short = model_name.split('/')[-1].split('.')[0]
local_rank = 0
# torch.cuda.set_device(local_rank)
# torch.manual_seed(0)

NClasses = 1
nf = 96
verbose = True
model_config = {'in_channels': 4,
                'spatial_size_init': (128, 128),
                'depths': [2, 2, 5, 2],
                'nfilters_init': nf,
                'nheads_start': nf // 4,
                'NClasses': NClasses,
                'verbose': verbose,
                'segm_act': 'sigmoid'}

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available():
    modeli = ptavit3d_dn(**model_config).to(device)
    modeli.load_state_dict(torch.load(model_name))
    model = modeli.to(device) # Set model to gpu
    model.eval()
    
# else:
#     torch.set_num_threads(40)
#     model = ptavit3d_dn(**model_config).to(device)
#     model.load_state_dict(torch.load(model_name, map_location=torch.device("cpu")))  
#     model.eval()


vdata = RocksDBDataset(dataFolder + 'output/rocks_db/FORCE.db/train.db', transform=TrainingTransform_for_rocks_Valid)

#vdata = VALIDataset()
preds = []

valid_loader = DataLoader(dataset=vdata, batch_size=1, shuffle=False, num_workers=0, pin_memory=True)

trg =  sorted(glob.glob(os.path.join(dataFolder + 'ai4boundaries/sentinel2/masks/' + 'LU' + '/*.tif')))

for i, (image, label) in enumerate(valid_loader):
    print(image.shape)
    print(type(image))
    
    if torch.cuda.is_available():
        with torch.no_grad():
            pred = model(image.cuda())
            preds.append(pred.detach().cpu().numpy())
    else:
        image = image.to(device)  # Move image to the correct device
        with torch.no_grad():
            pred = model(image)
            preds.append(pred.detach().cpu().numpy())